In [4]:

example = "Bonjour, je m'appelle Emad et j'ahbit à Paris"

from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-french")

sentence = Sentence(example)

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)




AttributeError: 'LSTM' object has no attribute '_flat_weights'

      score  index    word  start  end entity
0  0.999105      5   Sarah     17   22    PER
1  0.999755     10  Canada     36   42    LOC
2  0.998341     14   Ahmad     58   63    PER


,entity,word,score,index,start,end
0,PER,Sarah,0.999105,5,17,22
1,LOC,Canada,0.999755,10,36,42
2,PER,Ahmad,0.998341,14,58,63


In [69]:

# print resulting dataframe
print(count_df)

  entity  count
0    LOC      1
1    PER      2
